# REALM models after fine-tuning 

In [1]:
! mkdir download
! gsutil -m cp -R gs://realm-data/orqa_nq_model_from_realm/export/best_default/checkpoint/ download
! ls download/checkpoint/

Copying gs://realm-data/orqa_nq_model_from_realm/export/best_default/checkpoint/model.ckpt-300000.data-00000-of-00002...
Copying gs://realm-data/orqa_nq_model_from_realm/export/best_default/checkpoint/model.ckpt-300000.data-00001-of-00002...
Copying gs://realm-data/orqa_nq_model_from_realm/export/best_default/checkpoint/model.ckpt-300000.meta...
Copying gs://realm-data/orqa_nq_model_from_realm/export/best_default/checkpoint/model.ckpt-300000.index...
\ [4/4 files][  2.6 GiB/  2.6 GiB] 100% Done  64.7 MiB/s ETA 00:00:00           
Operation completed over 4 objects/2.6 GiB.                                      
model.ckpt-300000.data-00000-of-00002  model.ckpt-300000.index
model.ckpt-300000.data-00001-of-00002  model.ckpt-300000.meta


In [2]:
import tensorflow.compat.v1 as tf
import os

best_model_dir="download/checkpoint"

best_checkpoint_pattern = os.path.join(best_model_dir, "*.index")
best_checkpoint = tf.io.gfile.glob(best_checkpoint_pattern)[0][:-len(".index")]

best_model_REALM_NQ=tf.train.load_checkpoint(best_checkpoint)

In [3]:
variables_question_and_reader_encoder=[i for i in best_model_REALM_NQ.get_variable_to_dtype_map()]
variables_question_and_reader_encoder=sorted(variables_question_and_reader_encoder)

In [4]:
print("Number of variables =",len(variables_question_and_reader_encoder))

Number of variables = 1231


In [5]:
variables_question_and_reader_encoder[900:910] #dummy example

['reader/module/bert/encoder/layer_3/attention/self/key/bias',
 'reader/module/bert/encoder/layer_3/attention/self/key/bias/adam_m',
 'reader/module/bert/encoder/layer_3/attention/self/key/bias/adam_v',
 'reader/module/bert/encoder/layer_3/attention/self/key/kernel',
 'reader/module/bert/encoder/layer_3/attention/self/key/kernel/adam_m',
 'reader/module/bert/encoder/layer_3/attention/self/key/kernel/adam_v',
 'reader/module/bert/encoder/layer_3/attention/self/query/bias',
 'reader/module/bert/encoder/layer_3/attention/self/query/bias/adam_m',
 'reader/module/bert/encoder/layer_3/attention/self/query/bias/adam_v',
 'reader/module/bert/encoder/layer_3/attention/self/query/kernel']

In [6]:
variables_question_and_reader_encoder[40] #It should be a question encoder variable

'module/module/module/module/bert/encoder/layer_0/attention/self/key/bias'

In [7]:
variables_question_and_reader_encoder[660] #It should be a reader encoder variable

'reader/module/bert/encoder/layer_0/attention/self/key/bias'

# Comparison between question and document encoder

In [8]:
#Download question encoder
! gsutil -m cp -R gs://realm-data/cc_news_pretrained/embedder/ download

Copying gs://realm-data/cc_news_pretrained/embedder/saved_model.pb...
Copying gs://realm-data/cc_news_pretrained/embedder/assets/vocab.txt...
Copying gs://realm-data/cc_news_pretrained/embedder/tfhub_module.pb...
Copying gs://realm-data/cc_news_pretrained/embedder/variables/variables.data-00000-of-00001...
Copying gs://realm-data/cc_news_pretrained/embedder/encoded/checkpoint...
Copying gs://realm-data/cc_news_pretrained/embedder/encoded/encoded.ckpt.data-00000-of-00001...
Copying gs://realm-data/cc_news_pretrained/embedder/variables/variables.index...
Copying gs://realm-data/cc_news_pretrained/embedder/encoded/encoded.ckpt.meta...
Copying gs://realm-data/cc_news_pretrained/embedder/encoded/encoded.ckpt.index...
/ [9/9 files][  6.9 GiB/  6.9 GiB] 100% Done  57.5 MiB/s ETA 00:00:00           
Operation completed over 9 objects/6.9 GiB.                                      


In [9]:
! ls download/embedder

assets	encoded  saved_model.pb  tfhub_module.pb  variables


In [10]:
embedder_model_path="download/embedder"
document_embedder_model = tf.saved_model.load_v2(embedder_model_path, tags={})

In [11]:
variables_document_encoder = [x.name for x in document_embedder_model.variables]
variables_document_encoder = sorted(variables_document_encoder)

In [12]:
print("Number of variables =",len(variables_document_encoder))

Number of variables = 208


In [13]:
variables_document_encoder[23]

'module/module/module/bert/encoder/layer_0/output/dense/bias:0'

## We may compare the values for layer 0 output dense bias of BERT architecture, both for question and document encoder.

In [14]:
emb_document_encoder=[el for el in document_embedder_model.variables if el.name == 'module/module/module/bert/encoder/layer_0/output/dense/bias:0']
emb_document_encoder=emb_document_encoder[0].numpy()
emb_document_encoder.shape

(768,)

In [15]:
emb_question_encoder=best_model_REALM_NQ.get_tensor('module/module/module/module/bert/encoder/layer_0/attention/self/key/bias')
emb_question_encoder.shape

(768,)

In [16]:
emb_document_encoder[:5] #only pre-train

array([-0.06116943,  0.20037939,  0.0269921 ,  0.03042587,  0.04369077],
      dtype=float32)

In [17]:
emb_question_encoder[:5] #pre-train + fine-tuning. Indeed, the values are slightly different (I think due to fine-tuning)

array([ 0.00263759, -0.00765959,  0.000136  , -0.00216489,  0.01256373],
      dtype=float32)